In [2]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests, sys, json
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from bs4 import BeautifulSoup

### web scraping (escolher condição e nº de genes)

In [360]:
query= input('escolha o que quer pesquisar: ')
def url_get(i):
    url_list= [ ]
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format( query )
    url_list.append(url)
    return url_list
url_get(1)

content = []
for url in url_get(1):
    r = requests.get(url)
    content.append(r.content)
#print(content)

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(c, 'html.parser')
    a= soup.get_text()
#     print (a)

exist = re.findall(r"ID:\s+\d*(?=\D)", a, re.DOTALL)

c= ', '.join(exist)
h= c.replace('ID: ','')
IDS= h.split(', ')

n_genes= IDS[0:int(input('escolha o nº de genes que quer obter (máximo 20) atenção que o reultado pode ser menor que o desejado: '))] #can be more. related to ncbi pagination
print(n_genes)

if n_genes == ['']:
    print ("Não encontramos resultado para a sua pesquisa")
# else:
#     print (n_genes)

escolha o que quer pesquisar: diabetes
escolha o nº de genes que quer obter (máximo 20) atenção que o reultado pode ser menor que o desejado: 20
['16847', '1493', '7422', '3569', '1029', '5468', '387082', '79068', '3630', '6934', '170801', '3481', '3784', '26191', '170802', '3559', '3667', '3172', '6927', '3375']


### Buscar Links dos genes:

In [357]:
#não usamos
def url_get_id(i):
    url_list= [ ]
    for id in n_genes:
        url = "https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id )
        url_list.append(url)
    return url_list
url_get_id(n_genes) 

['https://www.ncbi.nlm.nih.gov/nuccore/424122',
 'https://www.ncbi.nlm.nih.gov/nuccore/29789',
 'https://www.ncbi.nlm.nih.gov/nuccore/26245415',
 'https://www.ncbi.nlm.nih.gov/nuccore/20918',
 'https://www.ncbi.nlm.nih.gov/nuccore/24563533',
 'https://www.ncbi.nlm.nih.gov/nuccore/39874648',
 'https://www.ncbi.nlm.nih.gov/nuccore/4839978',
 'https://www.ncbi.nlm.nih.gov/nuccore/42049400',
 'https://www.ncbi.nlm.nih.gov/nuccore/42048491',
 'https://www.ncbi.nlm.nih.gov/nuccore/36516274',
 'https://www.ncbi.nlm.nih.gov/nuccore/35606568',
 'https://www.ncbi.nlm.nih.gov/nuccore/35598500',
 'https://www.ncbi.nlm.nih.gov/nuccore/24564880',
 'https://www.ncbi.nlm.nih.gov/nuccore/24564814',
 'https://www.ncbi.nlm.nih.gov/nuccore/13446414']

### Buscar ids do ncbi:

In [361]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Ids.append(info.id)
print(len(Ids), Ids)

11 ['Z17674.1', 'V00883.1', 'X67017.1', 'X61243.1', 'M90830.1', 'Z15032.1', 'V01310.1', 'M94340.1', 'X54964.1', 'Z19157.1', 'X05328.1']


### Buscar quantidade de cds e a sua localização:

In [70]:
#original
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
featcds = [ ]
position=0
cds_location=[]
for info in records:
#     print(info.id)
    for i in range(len(info.features)):
        if info.features[i].type == "CDS":
            print(info.features[i].type == "CDS")
            featcds.append(i)
            cds=i
            a=str(cds)
#             print (len(a))
            position=i
            cds_location.append(info.features[position].location)
print(cds_location)
            
# print (pos)

True
True
[FeatureLocation(ExactPosition(145), ExactPosition(949), strand=1), FeatureLocation(BeforePosition(0), AfterPosition(92), strand=1)]


In [39]:
Ids.append('NG_009030.2')

In [362]:
#final
from Bio.SeqFeature import CompoundLocation
def get_CDS_location(id_list):
    database = 'nucleotide'
    email= 'rodrigoce9@gmail.com'
    Entrez.email = email
    handle = Entrez.efetch(db=database, id=id_list, rettype="gb") 
    records = list(SeqIO.parse(handle,"gb"))
    handle.close()
    cds_location_list = []
    several_location = []
    for info in records:
        for i in range(len(info.features)):
            if info.features[i].type == "CDS":
                cds_location = []
                cds_location.append(info.id)
                if isinstance(info.features[i].location, CompoundLocation):
                    for sub_location in info.features[i].location.parts:
                        several_location.append("[{} : {}]".format(sub_location.start, sub_location.end))
                    cds_location.append(several_location)
                else:
                    cds_location.append("[{} : {}]".format(info.features[i].location.start, info.features[i].location.end))
                cds_location_list.append(cds_location)
    return cds_location_list
for i in get_CDS_location(Ids):
    print(i)

# < indicates 5' to 3' (forward direction)
# < indicates 3' to 5' (reverse direction)  one of the sugestion to improve work ++++++++++++++++++++++++++++++++

['V00883.1', ['[223 : 316]', '[440 : 662]', '[1479 : 1608]', '[1077 : 1206]', '[1299 : 1359]', '[1460 : 1652]', '[1955 : 2632]', '[2996 : 3173]', '[3220 : 3314]', '[3557 : 3713]', '[3759 : 4226]', '[4474 : 4558]', '[4927 : 5093]', '[5604 : 5666]', '[5744 : 5837]', '[5885 : 6060]', '[11965 : 12069]', '[12179 : 12279]', '[10518 : 10605]', '[9659 : 10043]', '[9482 : 9611]', '[9157 : 9434]', '[8593 : 9110]', '[8461 : 8540]', '[8174 : 8408]', '[7810 : 8097]', '[7509 : 7723]', '[6491 : 7055]', '[6320 : 6440]', '[6211 : 6271]', '[18977 : 19035]', '[18483 : 18569]', '[18040 : 18433]', '[17645 : 17973]', '[17229 : 17598]', '[17059 : 17182]', '[16839 : 17013]', '[16317 : 16791]', '[15591 : 15953]', '[15410 : 15546]', '[15063 : 15361]', '[14610 : 14807]', '[14264 : 14561]', '[14026 : 14212]', '[13481 : 13976]', '[13171 : 13353]', '[12886 : 13124]', '[12746 : 12826]', '[25527 : 25691]', '[25244 : 25443]', '[25117 : 25190]', '[24440 : 24847]', '[23979 : 24232]', '[23558 : 23921]', '[23361 : 23484]'

### Buscar Ids de proteinas

In [123]:
#original
list_pro=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    for i in info.features:
        if i.type == "CDS":
            pro= str(i.qualifiers["protein_id"])
            list_pro.append(pro)
div= ', '.join(list_pro)
h= div.replace("['",'')
hh= h.replace("']",'')
ID_PROT= hh.split(', ')
print (ID_PROT) # is used below in Uniprot

['CAA24252.1', 'CAA47407.1']


In [363]:
#FINAL
result_dict = {}
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist=Ids
# idlist= Ids.remove('U49845')
for ids in idlist:
    list_pro=[]
    handle = Entrez.efetch(db=database, id=ids, rettype="gb") 
    records = list(SeqIO.parse(handle,"gb"))
    handle.close()
    for info in records:
        list_pro.append(info.id)
        for i in info.features:
            if i.type == "CDS":
                pro= str(i.qualifiers["protein_id"])
                list_pro.append(pro)
        if len(list_pro)==1: #if no protein_id was found
            result_dict[info.id] = "N/A"
        else:
            list_pro = [x.replace("['",'').replace("']",'') for x in list_pro]
            result_dict[info.id] = ', '.join(list_pro[1:])
print(result_dict)
#key: id_genebank values: id_Uniprot

{'Z17674.1': 'N/A', 'V00883.1': 'CAA24252.1', 'X67017.1': 'CAA47407.1', 'X61243.1': 'N/A', 'M90830.1': 'N/A', 'Z15032.1': 'CAA78750.1', 'V01310.1': 'CAA24617.1', 'M94340.1': 'N/A', 'X54964.1': 'CAA38712.1', 'Z19157.1': 'CAB63418.2, CAE18055.3, CAA79568.2, CAA79570.2, CAA79569.3, CAA79566.3', 'X05328.1': 'CAA28945.1'}


In [306]:
#TESTE
# lista =[]
# database = 'nucleotide'
# email= 'rodrigoce9@gmail.com'

# list_pro=[]
# handle = Entrez.efetch(db=database, id='U49845', rettype="gb") 
# records = list(SeqIO.parse(handle,"gb"))
# handle.close()
# for info in records:
#     list_pro.append(info.id)
#     for i in info.features:
#         if i.type == "CDS":
#             pro= str(i.qualifiers["protein_id"])
#             list_pro.append(pro)

# div= ', '.join(list_pro)
# h= div.replace("['",'')
# hh= h.replace("']",'')
# ID_PROT= hh.split(', ')
# lista.append(ID_PROT)
# print(lista) 

### Buscar Links de proteinas:

In [364]:
#final
def url_get_id_p(dic):
    url_list_p=[ ]
    for key, value in dic.items():
        if dic[key] !='N/A':
            url_id_p= "https://www.uniprot.org/uniprotkb?query={}".format(value)
            url_list_p.append(url_id_p)
    return url_list_p
url_ids_protein=url_get_id_p(result_dict)
print(url_ids_protein)  # but we don't need to save this links

['https://www.uniprot.org/uniprotkb?query=CAA24252.1', 'https://www.uniprot.org/uniprotkb?query=CAA47407.1', 'https://www.uniprot.org/uniprotkb?query=CAA78750.1', 'https://www.uniprot.org/uniprotkb?query=CAA24617.1', 'https://www.uniprot.org/uniprotkb?query=CAA38712.1', 'https://www.uniprot.org/uniprotkb?query=CAB63418.2, CAE18055.3, CAA79568.2, CAA79570.2, CAA79569.3, CAA79566.3', 'https://www.uniprot.org/uniprotkb?query=CAA28945.1']


### Buscar Links de proteinas na Uniprot:

In [370]:
#final
def url_get_id_p(string):
#     print(string)
    url_list_p=[ ]
    url_id_p= "https://www.uniprot.org/uniprotkb/{}/entry".format(str(string))
    url_list_p.append(url_id_p)
    return ''.join(url_list_p)
print(url_ids_protein)


https://www.uniprot.org/uniprotkb/P02099/entry


In [323]:
for index, value in enumerate(get_list_uniprot(ID_PROT)):
    print(get_list_uniprot(ID_PROT)[index][0])
#     print(get_list_uniprot(ID_PROT)[index][1])
    

P02099
Q00873


### Buscar id proteina (uniprot):

In [366]:
#final
def get_url(url, **kwargs):
    response = requests.get(url, **kwargs);
    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()
    return response

def get_list_uniprot(ID_PROT):
    results = {}
    for i in ID_PROT:
        result = {}
        for field in fields:
            result[field] = get_field_for_id(i, field)
        results[i] = result
    uniprot_final_list=[]
    for key, value in results.items():
        uniprot_list=[]
        for x in value.values():
            entry = re.search(r'b\'Entry\\n(.+?(?=\\n\'))', x, re.DOTALL)
            function = re.search( r'b\'Function \[CC\]\\n.{9} (.+?(?=\\n\'))', x, re.DOTALL )
            location_exist = re.search( r'b\'Subcellular location \[CC\]\\nSUBCELLULAR LOCATION: (.+?(?=\\n\'))', x, re.DOTALL )
            location_notexist = re.search( r'b\'Subcellular location \[CC\]\\n\\n\'', x, re.DOTALL )   
            sequence = re.search(r'b\'Sequence\\n(.+?(?=\\n\'))', x, re.DOTALL)
            if entry:
                ent=entry.group(1)
                uniprot_list.append(entry.group(1))
            if function:
                uniprot_list.append(function.group(1))
            if location_exist:
                uniprot_list.append(location_exist.group(1))
            if location_notexist:
                uniprot_list.append("N/A")
            if sequence:
                uniprot_list.append(sequence.group(1))
                uniprot_list.append(len(sequence.group(1)))
                uniprot_list.append(url_get_id_p(ent))
            
        uniprot_final_list.append(uniprot_list)
#         uniprot_final_list.append(url_get_id_p(ent))
        
    return uniprot_final_list
print(get_list_uniprot(ID_PROT)) #list of lists (ID_Uniprot, Function, Subcelular location, Protein seq, length_aa)

[['P02099', 'N/A', 'This protein functions as an embryonic globin, but the gene structure and chromosomal location resemble more closely the human gamma chain gene, which codes for a fetal globin.', 'MVHFTAEEKAAITSTWKLVDVEDAGAEALGRLLVVYPWTQRFFDSFGNLSSSSAIMGNPKVKAHGKKVLTAFGDAVKNVDDLKNTFAHLSELHCDRLHVDPENFKLLGNVLVIVLAKYFGKEFTPQVQSAWQKLVAGVATALAHKYH', 147, 'https://www.uniprot.org/uniprotkb/P02099/entry'], ['Q00873', 'Mitochondrion inner membrane {ECO:0000269|PubMed:1499554}. Note=Partially exposed to the intermembrane space. {ECO:0000305|PubMed:1499554}.', 'Lyase that catalyzes the covalent linking of the heme group to the cytochrome C1 apoprotein to produce the mature functional cytochrome. {ECO:0000269|PubMed:1499554}.', 'MMSSDQQGKCPVDEETKKLWLREHGNEAHPGATAPGNQLECSANPQDNDKTPEYHTTVDLSQSREVSTIPRTNSDRNWIYPSEKQFYEAMMKKNWDPNSDDMKVVVPLHNSINERVWNYIKSWEDKQGGEACGGIKLTNFKGDSKKLTPRAWFRSRILHLAKPFDRHDWQIDRCGKTVDYVIDFYSTDLNDANSQQQPLIYLDVRPKLNSFEGFRLRFWKSLGF', 224, 'https://www.uniprot.org/uniprotkb/Q0

### Buscar seq dos genes:

In [ ]:
#Biopython
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    print(info.id)
    print(info.seq[0:50],'...',info.seq[-10:])
    print()
    seq.append(f'{info.seq[0:50]}...{info.seq[-10:]}')
    

### Buscar artigos:

In [ ]:
Ids=[]
database = 'PubMed'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="medline", retmode="text") 
records = Medline.parse(handle)
description=[]
for info in records:
    print("title:", info.get("TI", "-"))
    print('authors: ', info.get("AU", "-"))
    print("source:", info.get("SO", "-"))
    print("affiliation: ", info.get("AD", "-") ) # FAZER O IF/ELSE PARA O CASO DE NÃO TER AFFILIATION
    print()

### Começar povoar

In [52]:
#para povoar CDS
print(result_dict)
print(Ids)
# print(get_CDS_location(Ids)[0])
#key: id_genebank values: id_Uniprot
for i in get_CDS_location(Ids):
    print(i)

{'X03907.1': 'CAA27540.1', 'AA928418.1': 'N/A', 'AAADR0035516.1': 'N/A', 'L20655.1': 'AAA47844.1', 'AAADU0045050.1': 'N/A', 'NG_009030.2': 'NP_001361282.1'}
['X03907.1', 'AA928418.1', 'AAADR0035516.1', 'L20655.1', 'AAADU0045050.1', 'NG_009030.2']
['X03907.1', '[145 : 949]']
['L20655.1', '[<0 : >92]']
['NG_009030.2', ['[66454 : 66602]', '[196494 : 196600]', '[220376 : 220579]', '[224378 : 224564]', '[237847 : 238049]', '[239526 : 239740]', '[241074 : 241197]', '[244108 : 244243]', '[247567 : 247717]', '[251849 : 252014]', '[253604 : 253877]', '[255411 : 255748]', '[257341 : 259809]']]


In [51]:
#para povoar CDS

final_list_povo=[] #(idGenebank, CDS location, idUniprot)
list_CDS = []
for key in result_dict:
    list_CDS = []
    list_CDS.append(key)
#     print(key)
    for index in get_CDS_location(Ids):
        if key == index[0]:
            list_CDS.append(index[1])
            list_CDS.append(result_dict[key])
            final_list_povo.append(list_CDS)
#         print(index[0])
print(final_list_povo[0])

['X03907.1', '[145 : 949]', 'CAA27540.1']


In [ ]:
#Povoação "CDS"  #incompleto, muito

import mysql.connector as SQLC
import mysql.connector


DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

Cursor = DataBase.cursor()
try:
    for index, value in enumerate(Ids):
        #print(value)
        #print(des[index])
        sql= "INSERT INTO CDS (Location, ID_genebank, ID_Uniprot) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        val=(get_CDS_location(Ids)[1], final_list_povo)
#         print(type(des[index]))
        Cursor.execute(sql,val)
#         for values in des:
#             Cursor.execute(f"INSERT INTO Gene ( Description)  VALUES ('{values}')")
except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

In [367]:
#mini testes
# print(get_list_uniprot(ID_PROT)) #list of lists (ID_Uniprot, Function, Subcelular location, Protein seq, length_aa)
print(get_list_uniprot(ID_PROT)[0][0])
for index, value in enumerate(get_list_uniprot(ID_PROT)):
    print(str(get_list_uniprot(ID_PROT)[index][5]))

P02099
https://www.uniprot.org/uniprotkb/P02099/entry
https://www.uniprot.org/uniprotkb/Q00873/entry


In [368]:
#Povoação "Uniprot"
# print(uniprot_final_list) #(ID_Uniprot, Subcelular location, Function, Protein seq, length_aa)

import mysql.connector as SQLC
import mysql.connector


DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)
DataBase.autocommit = True # allows the change be done

# url_ids_protein=url_get_id_p(get_list_uniprot(ID_PROT)[0][0])
# print(url_ids_protein)
Cursor = DataBase.cursor()
try:
    for index, value in enumerate(get_list_uniprot(ID_PROT)):
        #print(value)
        sql= "INSERT INTO Uniprot (ID_Uniprot, Subcelular_Location, Function, Protein_sequence, length_aa, Link_Uniprot) VALUES (%s, %s, %s, %s, %s, %s)"
        val=(get_list_uniprot(ID_PROT)[index][0],get_list_uniprot(ID_PROT)[index][1], get_list_uniprot(ID_PROT)[index][2], get_list_uniprot(ID_PROT)[index][3], get_list_uniprot(ID_PROT)[index][4], get_list_uniprot(ID_PROT)[index][5])
        Cursor.execute(sql,val)

except mysql.connector.Error as e:
    print("Erro na escrita na base de dados: {}".format(e) )    
finally:
    DataBase.close()

Erro na escrita na base de dados: 1062 (23000): Duplicate entry 'P02099' for key 'PRIMARY'


In [ ]:
import pandas as pd 

DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="AP_db_KRG"
)

####### end of connection ####
tabela = pd.read_sql("Select * FROM Gene", DataBase)
DataBase.close()
print(tabela)